In [1]:
import polars

from autofeat.dataset import *
from autofeat.transform import *

In [2]:
datasets = [
    KaggleDataset(id="mahmoudemadabdallah/hr-analytics-employee-attrition-and-performance"),
]

In [3]:
tables = [
    table
    for dataset in datasets
    for table in dataset.list_tables()
]

In [4]:
transform = (
    Cast()
    .then(Encode())
    .then(Aggregate(by={"EmployeeID"}))
    .then(Identity(), Join(on={"EmployeeID"}))
    .then(Identity(), Combine())
)

In [5]:
results = polars.collect_all([
    table.data.select(
        (polars.selectors.boolean() | polars.selectors.numeric())
        .name.suffix(f" from {table.name}")
    )
    for table in transform.apply(tables)
])

In [6]:
features = polars.concat(
    [df for df in results if len(df) == 1],
    how="horizontal",
)

In [7]:
features

count(*) from RatingLevel.csv,max(RatingID) from RatingLevel.csv,mean(RatingID) from RatingLevel.csv,median(RatingID) from RatingLevel.csv,min(RatingID) from RatingLevel.csv,std(RatingID) from RatingLevel.csv,sum(RatingID) from RatingLevel.csv,var(RatingID) from RatingLevel.csv,count(*) from PerformanceRating.csv,max(JobSatisfaction) from PerformanceRating.csv,mean(JobSatisfaction) from PerformanceRating.csv,median(JobSatisfaction) from PerformanceRating.csv,min(JobSatisfaction) from PerformanceRating.csv,std(JobSatisfaction) from PerformanceRating.csv,sum(JobSatisfaction) from PerformanceRating.csv,var(JobSatisfaction) from PerformanceRating.csv,max(RelationshipSatisfaction) from PerformanceRating.csv,mean(RelationshipSatisfaction) from PerformanceRating.csv,median(RelationshipSatisfaction) from PerformanceRating.csv,min(RelationshipSatisfaction) from PerformanceRating.csv,std(RelationshipSatisfaction) from PerformanceRating.csv,sum(RelationshipSatisfaction) from PerformanceRating.csv,var(RelationshipSatisfaction) from PerformanceRating.csv,max(EnvironmentSatisfaction) from PerformanceRating.csv,mean(EnvironmentSatisfaction) from PerformanceRating.csv,median(EnvironmentSatisfaction) from PerformanceRating.csv,min(EnvironmentSatisfaction) from PerformanceRating.csv,std(EnvironmentSatisfaction) from PerformanceRating.csv,sum(EnvironmentSatisfaction) from PerformanceRating.csv,var(EnvironmentSatisfaction) from PerformanceRating.csv,max(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,mean(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,median(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,min(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,std(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,sum(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,var(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,…,median(DistanceFromHome (KM)) / std(StockOptionLevel) from Employee.csv,mean(YearsWithCurrManager) + count(*) from Employee.csv,mean(YearsWithCurrManager) - count(*) from Employee.csv,count(*) - mean(YearsWithCurrManager) from Employee.csv,mean(YearsWithCurrManager) * count(*) from Employee.csv,mean(YearsWithCurrManager) / count(*) from Employee.csv,count(*) / mean(YearsWithCurrManager) from Employee.csv,mean(YearsWithCurrManager) + std(YearsAtCompany) from Employee.csv,mean(YearsWithCurrManager) - std(YearsAtCompany) from Employee.csv,std(YearsAtCompany) - mean(YearsWithCurrManager) from Employee.csv,mean(YearsWithCurrManager) * std(YearsAtCompany) from Employee.csv,mean(YearsWithCurrManager) / std(YearsAtCompany) from Employee.csv,std(YearsAtCompany) / mean(YearsWithCurrManager) from Employee.csv,mean(YearsWithCurrManager) + median(DistanceFromHome (KM)) from Employee.csv,mean(YearsWithCurrManager) - median(DistanceFromHome (KM)) from Employee.csv,median(DistanceFromHome (KM)) - mean(YearsWithCurrManager) from Employee.csv,mean(YearsWithCurrManager) * median(DistanceFromHome (KM)) from Employee.csv,mean(YearsWithCurrManager) / median(DistanceFromHome (KM)) from Employee.csv,median(DistanceFromHome (KM)) / mean(YearsWithCurrManager) from Employee.csv,count(*) + std(YearsAtCompany) from Employee.csv,count(*) - std(YearsAtCompany) from Employee.csv,std(YearsAtCompany) - count(*) from Employee.csv,count(*) * std(YearsAtCompany) from Employee.csv,count(*) / std(YearsAtCompany) from Employee.csv,std(YearsAtCompany) / count(*) from Employee.csv,count(*) + median(DistanceFromHome (KM)) from Employee.csv,count(*) - median(DistanceFromHome (KM)) from Employee.csv,median(DistanceFromHome (KM)) - count(*) from Employee.csv,count(*) * median(DistanceFromHome (KM)) from Employee.csv,count(*) / median(DistanceFromHome (KM)) from Employee.csv,median(DistanceFromHome (KM)) / count(*) from Employee.csv,std(YearsAtCompany) + median(DistanceFromHome (KM)) from Employee.csv,std(YearsAtCompany) - median(DistanceFromHome (KM)) from Employee.csv,median(DistanceFromHome (KM)) - 